<a href="https://colab.research.google.com/github/ampig2020/Data-Science-Hands-on-Practice/blob/main/Finding_optimal_locations_of_new_stores_using_Decision_Optimization_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how Decision Optimization can help to prescribe decisions for a complex constrained problem using CPLEX Modeling for Python to help determine the optimal location for a new store. This notebook requires the Commercial Edition of CPLEX engines, which is included in the latest Python XS + DO environment in Watson Studio.

**Finding Optimal Locations for New Stores**

This notebook is an example of how Decision Optimization can help to prescribe decisions for a complex constrained problem.

When you finish this notebook, you'll have a foundational knowledge of Prescriptive Analytics.

This notebook requires the Commercial Edition of CPLEX engines, which is included in the Default Python 3.7 XS + DO in Watson Studio.

Table of contents:

Describe the business problem


How decision optimization (prescriptive analytics) can help

Use decision optimization

Step 1: Import the docplex package

Step 2: Model the data

Step 3: Prepare the data

Step 4: Set up the prescriptive model

Define the decision variables

Express the business constraints

Express the objective

Solve with the Decision Optimization solve service

Step 5: Investigate the solution and run an example analysis

Summary

---



**Describe the business problem**

A fictional Coffee Company plans to open N shops in the near future and needs to determine where they should be located knowing the following:

Most of the customers of this coffee brewer enjoy reading and borrowing books, so the goal is to locate those shops in such a way that all the city public libraries are within minimal walking distance.

We use Chicago open data for this example.

We implement a K-Median model to get the optimal location of our future shops.

**How decision optimization can help**

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes. It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.

Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.

Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.


With prescriptive analytics, you can:

Automate the complex decisions and trade-offs to better manage your limited resources.
Take advantage of a future opportunity or mitigate a future risk.
Proactively update recommendations based on changing events.
Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

***Use decision optimization***

Step 1: Import the docplex package


This package is presintalled on Watson Studio.

In [ ]:
import sys
import docplex.mp

*Note that the more global package docplex contains another subpackage docplex.cp that is dedicated to Constraint Programming, another branch of optimization.*

**Step 2: Model the data**

The data for this problem is quite simple: it is composed of the list of public libraries and their geographical locations.

The data is acquired from Chicago open data as a JSON file, which is in the following format: 
data" : [ [ 1, "13BFA4C7-78CE-4D83-B53D-B57C60B701CF", 1, 1441918880, "885709", 1441918880, "885709", null, "Albany Park", "M, W: 10AM-6PM;  TU, TH: 12PM-8PM; F, SA: 9AM-5PM; SU: Closed", "Yes", "Yes ", "3401 W. Foster Avenue", "CHICAGO", "IL", "60625", "(773) 539-5450", [ "http://www.chipublib.org/locations/1/", null ], [ null, "41.975456", "-87.71409", null, false ] ]

This code snippet represents library "3401 W. Foster Avenue" located at 41.975456, -87.71409

**Step 3: Prepare the data**

We need to collect the list of public libraries locations and keep their names, latitudes, and longitudes.

In [ ]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

Define how to compute the earth distance between 2 points:

To easily compute distance between 2 points, we use the Python package geopy

In [ ]:
try:
    import geopy.distance
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install geopy 
    else:
        !pip install --user geopy

In [ ]:
# Simple distance computation between 2 locations.
from geopy.distance import great_circle
 
def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

Declare the list of libraries

Parse the JSON file to get the list of libraries and store them as Python elements.

In [ ]:
def build_libraries_from_url(url, name_pos, lat_long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[name_pos]
        try:
            latitude = float(location[lat_long_pos][1])
            longitude = float(location[lat_long_pos][2])
        except TypeError:
            latitude = longitude = None
        try:
            name = str(uname)
        except:
            name = "???"
        name = "P_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

In [ ]:
libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD',
                                   name_pos=10,
                                   lat_long_pos=16)

In [ ]:
print("There are %d public libraries in Chicago" % (len(libraries)))

There are 81 public libraries in Chicago

Define number of shops to open

Create a constant that indicates how many coffee shops we would like to open.

In [ ]:
nb_shops = 5
print("We would like to open %d coffee shops" % nb_shops)

We would like to open 5 coffee shops


Validate the data by displaying them

We will use the folium library to display a map with markers.

In [ ]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install folium

In [ ]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

After running the above code, the data is displayed but it is impossible to determine where to ideally open the coffee shops by just looking at the map.


Let's set up DOcplex to write and solve an optimization model that will help us determine where to locate the coffee shops in an optimal way.

**Step 4: Set up the prescriptive model**

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.7.9, located at: /opt/conda/envs/Python-3.7-main/bin/python
* docplex is present, version is 2.14.186
* CPLEX library is present, version is 12.10.0.0, located at: /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages
* pandas is present, version is 1.0.5

**Displaying the solution.**

Coffee shops are highlighted in red.

In [ ]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for coffeeshop in open_coffeeshops:
    lt = coffeeshop.y
    lg = coffeeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
    
for b in libraries:
    if b not in open_coffeeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(map_osm)
    

for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_child(folium.PolyLine(coordinates, color='#FF0000', weight=5))

map_osm

**Summary**

You have learned how to set up, formulate and solve an optimization model using Decision Optimization in Watson Studio.
